# Setup

In [0]:
import tensorflow as tf
import numpy as np
import os
import tensorflow.contrib.slim as slim
from google.colab import files
from matplotlib.pyplot import imshow
import numpy as np
from PIL import Image
import matplotlib.pyplot as plt
import cv2

%matplotlib inline

In [0]:
uploaded = files.upload()

In [0]:
tf.logging.set_verbosity(tf.logging.INFO)

TRAIN_FILE = 'facial_keypoints_train.txt'
EVAL_FILE = 'facial_keypoints_eval.txt'
LOG_DIR = 'log_dir'
IS_TRAINING = False
CHECKPOINT_PATH = None
CHECKPOINT_NUMBER=79
BATCH_SIZE = 32

# Analiza setului de date

In [0]:
with open(TRAIN_FILE, "r") as train_file:
  i = 0
  for line in train_file:
    if i == 5:
      break
    i += 1
    print(line)
  
    # TODO:
    # Afisati pentru primele 5 randuri Coordonatele punctelor si valorile pixelilor
    # Se va observa structura datelor in fisier: Fiecare rand contine un exemplu. Primele 30 de valori corespund coordonatelor x si y ale keypoints, iar urmatoarele 9216 valori sunt valorile pixelilor din imaginea flattened
    
    

In [0]:
with open(TRAIN_FILE, "r") as train_file:
  i = 0
  for line in train_file:
    if i == 5:
      break
    i += 1
    line=line.split()
    plt.figure()
    imshow(np.reshape(np.uint8(line[30:]),(96,96)),cmap="gray")
    # TODO: Completati codul pentru a afisa primele 5 imagini din setul de antrenare. Tineti cont de faptul ca o imagine in fisier este flattened, iar dimensiunea originala este 96*96 in format grayscale.
    # Hint: folositi imshow
    
    

In [0]:
with open(TRAIN_FILE, "r") as train_file:
  i = 0
  for line in train_file:
    if i == 5:
      break
    i += 1
    line = line.split()
    
    current=np.reshape(np.uint8(line[30:]),(96,96))
    plt.figure()
    for j in range(1,29,2):
      cv2.circle(current,(round(float(line[j+1])),round(float(line[j]))),3,255,1)
    imshow(current,cmap="gray")
    # TODO: Afisati cele 15 keypoints in primele 5 imagini din setul de antrenare. Primele 15 valori de pe fiecare rand au semnificatia keypoint_1_x, keypoint_1_y, keypoint_2_x, keypoint_2_y...
    # Hint: folositi cv2.circle
    
    

# Network

In [0]:
class Model(object):

    def __init__(self, inputs, labels, num_outputs, is_training=True):
        self.inputs = inputs
        self.labels = labels
        self.num_outputs = num_outputs

        self.learning_rate = 0.0002
        self.beta1 = 0.9
        self.is_training = is_training

        self.endpoints = {}

        self.create_graph()

        if self.is_training:
            self.create_loss()
            self.build_train_op()
            self.create_summaries()

    def create_graph(self):
        input = tf.reshape(self.inputs, shape=(-1, 96, 96, 1))
        endpoints = {}
        with tf.variable_scope("model_"):
          
            # TODO: Creati arhitectura retelei. Tineti cont de faptul ca input-ul este o imagine, iar rezultatul un vector de 30 de valori
            # Experimentati diverse arhitecturi si incercati sa obtineti un loss cat mai mic
            
            net = slim.conv2d(input, 32, [3,3], stride=1, padding ='SAME')
            net = slim.max_pool2d(net, [2, 2])
            net = slim.conv2d(net,64,[3,3],stride=1,padding='SAME')
            net = slim.flatten(net)
            net = slim.fully_connected(net, 30, activation_fn=None)

            self.endpoints["pred"] = net


    def create_loss(self):
        with tf.name_scope("losses"):
            pred = self.endpoints["pred"]
            self.total_loss = tf.losses.mean_squared_error(labels=self.labels, predictions=pred, scope="loss_reg")


    def create_summaries(self):
        summary_loss = tf.summary.scalar("total_loss", self.total_loss)
        summary_lr = tf.summary.scalar("learning_rate", self.optim._lr )
        self.summary_op = tf.summary.merge([summary_loss, summary_lr], name='summary_op')

    def build_train_op(self):
        with tf.name_scope('train_op'):
          self.optim = tf.train.AdamOptimizer(self.learning_rate, self.beta1)
          self.train_op = slim.learning.create_train_op(self.total_loss, self.optim)
          # self.global_step = tf.Variable(0, name='global_step', trainable=False)
          # self.incr_global_step = tf.assign(self.global_step, self.global_step + 1)


# Aux

In [0]:
def decode_csv(line):

    parsed_line = tf.decode_csv(line, [[0.] for _ in range(96 ** 2 + 30)],  field_delim=" ")
    label = parsed_line[:30]

    features = parsed_line[30:]
    return features, label
  

def make_initializable_iterator(dataset):
  """Creates an iterator, and initializes tables.

  This is useful in cases where make_one_shot_iterator wouldn't work because
  the graph contains a hash table that needs to be initialized.

  Args:
    dataset: A `tf.data.Dataset` object.

  Returns:
    A `tf.data.Iterator`.
  """
  iterator = dataset.make_initializable_iterator()
  tf.add_to_collection(tf.GraphKeys.TABLE_INITIALIZERS, iterator.initializer)
  return iterator

def _debug_tf_data(iterator):
    # debug the dataset
    batch_features, batch_labels = iterator.get_next()
    sess = tf.Session()
    sess.run(iterator.initializer)
    while (True):
        r = sess.run([batch_features, batch_labels])
        print(r)
        break

def restore_map():
    return {var.op.name: var for var in tf.global_variables()}

def get_variables_available_in_checkpoint(variables,
                                          checkpoint_path,
                                          include_global_step=True):
  if isinstance(variables, list):
    variable_names_map = {variable.op.name: variable for variable in variables}
  elif isinstance(variables, dict):
    variable_names_map = variables
  else:
    raise ValueError('`variables` is expected to be a list or dict.')
  ckpt_reader = tf.train.NewCheckpointReader(checkpoint_path)
  ckpt_vars_to_shape_map = ckpt_reader.get_variable_to_shape_map()
  if not include_global_step:
    ckpt_vars_to_shape_map.pop(tf.GraphKeys.GLOBAL_STEP, None)
    ckpt_vars_to_shape_map.pop('train_op/beta1_power', None)
    ckpt_vars_to_shape_map.pop('train_op/beta2_power', None)
    ckpt_vars_to_shape_map.pop('train_op/global_step', None)
  vars_in_ckpt = {}
  for variable_name, variable in sorted(variable_names_map.items()):
    if variable_name in ckpt_vars_to_shape_map:
      if ckpt_vars_to_shape_map[variable_name] == variable.shape.as_list():
        vars_in_ckpt[variable_name] = variable
      else:
        tf.logging.warning('Variable [%s] is available in checkpoint, but has an '
                        'incompatible shape with model variable. Checkpoint '
                        'shape: [%s], model variable shape: [%s]. This '
                        'variable will not be initialized from the checkpoint.',
                        variable_name, ckpt_vars_to_shape_map[variable_name],
                        variable.shape.as_list())
    else:
      tf.logging.warning('Variable [%s] is not available in checkpoint',
                      variable_name)
  if isinstance(variables, list):
    return vars_in_ckpt.values()
  return vars_in_ckpt

# Train

In [0]:
def train():
    file_path = TRAIN_FILE
    dataset = tf.data.TextLineDataset(file_path).map(decode_csv)

    dataset = dataset.shuffle(buffer_size=512).repeat().batch(BATCH_SIZE)
    iterator = make_initializable_iterator(dataset)
    batch_features, batch_labels = iterator.get_next()

    #_debug_tf_data(iterator)

    model = Model(inputs=batch_features, labels=batch_labels, num_outputs=30, is_training=True)
    train_tensor = model.train_op


    saver = tf.train.Saver(
        keep_checkpoint_every_n_hours=1, max_to_keep=20)

    if CHECKPOINT_PATH:
      var_map = restore_map()
      available_var_map = (get_variables_available_in_checkpoint(
          var_map, CHECKPOINT_PATH, include_global_step=False))
      init_saver = tf.train.Saver(available_var_map)

      def initializer_fn(sess):
          init_saver.restore(sess, CHECKPOINT_PATH)
      init_fn = initializer_fn
    else:
      init_fn = None

    slim.learning.train(
        train_tensor,
        init_fn=init_fn,
        logdir=LOG_DIR,
        summary_op=model.summary_op,
        number_of_steps=100000,
        save_summaries_secs=600,
        save_interval_secs=30,
        saver=saver)



# Eval

In [0]:
def eval(num_steps):
    file_path = EVAL_FILE
    dataset = tf.data.TextLineDataset(file_path).map(decode_csv)

    dataset = dataset.batch(1)
    iterator = dataset.make_initializable_iterator()
    batch_features, batch_labels = iterator.get_next()

    model = Model(inputs=batch_features, labels=batch_labels, num_outputs=30, is_training=False)
    errs = []
    saver = tf.train.Saver()
    with tf.Session() as sess:
        sess.run(iterator.initializer)
        saver.restore(sess, os.path.join(LOG_DIR, f"model.ckpt-{CHECKPOINT_NUMBER}"))
        for i in range(num_steps):
            feat, labels, preds = sess.run([batch_features, batch_labels, model.endpoints['pred']])
            err = np.abs(labels - preds)
            errs.extend(err[0])

        mae = np.mean(np.array(errs))

        print("MAE: : {}".format(mae))


# Main

In [0]:
tf.reset_default_graph()
if IS_TRAINING:
    train()
else:
    eval(num_steps=321)

# Download checkpoints

In [0]:
!zip -r logs.zip log_dir
files.download('logs.zip')

In [0]:
!ls log_dir


In [0]:
fis = open("test.txt","w")

In [0]:
!ls